# Query for Spectraction Results in OGA

 work with Weakly_2023_01
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/10/31
- last update : 2023/01/12


In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g754a7f0350+935911ccad 	current w_2023_01 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
import pickle

In [6]:
from astropy.time import Time
import astropy.units as u

## Config

### Butler

In [7]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

### Date & Filter & Disperser

In [8]:
# path index for each month
#DATE="20220912"
#DATE="20220913"
DATE="20221207"
filterdispersername = "empty~holo4_003"
#filterdispersername = "BG40~holo4_003"
#filterdispersername = "FELH0600~holo4_003"

### Spectractor

In [9]:
configmode = "PSF2DFFM"

### The collection

In [10]:
#my_collection = "u/dagoret/spectro/noflat/empty~holo4/"+str(DATE)
my_collection = "u/dagoret/BPS_manyspectro_v0" # january 12th 2023
datasetRefs = registry.queryDatasets(datasetType='spectraction', collections=my_collection, where= "instrument='LATISS'")

In [11]:
# following line not possible in OGA
#butler = butlerUtils.makeDefaultLatissButler(extraCollections=[my_collection])

## Loop on exposures

In [12]:
all_dataId = []     # full data id
all_spec = []       # spectra
all_exposures = []  # exposure number
all_num = []        # sequence numbers



for i, ref in enumerate(datasetRefs):

    print(f"============= ({i}) ============datasetType = spectraction ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_exposure = ref.dataId["visit"]
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0}
    print(the_dataId)
    
    # retrieve the spectrum from Butler
    #spec       = butler.get('spectraction',the_dataId)
    spec = butler.get('spectraction', visit=the_exposure, detector=0, collections=my_collection, instrument='LATISS')
    
    # save in collection lists
    all_dataId.append(the_dataId) 
    all_exposures.append(the_exposure)
    all_spec.append(spec)
    all_num.append(the_seq_num)
    if i > 10:
        break

============= (0) ============datasetType = spectraction ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2022120800269}
visit...................: 2022120800269
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/BPS_manyspectro_v0/20230112T090722Z
{'day_obs': 20221208, 'seq_num': 269, 'detector': 0}
============= (1) ============datasetType = spectraction ============================================
fullId..................: {band: 'white', instrument: 'LATISS', detector: 0, physical_filter: 'empty~holo4_003', visit: 2022120800116}
visit...................: 2022120800116
band....................: white
physical filter.........: empty~holo4_003
run.....................: u/dagoret/BPS_manyspectro_v0/20230112T090722Z
{'day_obs': 20221208, 'seq_num': 116, 'detector': 0}
============= (2) ============datasetType = spec

## Plot

In [13]:
FLAG_ORDER2 = False

infos = []
all_lambdas=[]
all_data=[]
all_data_err=[]

if FLAG_ORDER2:
    all_lambdas_order2=[]
    all_data_order2=[]
    all_data_err_order2=[]


idx=0
for spec in all_spec:
    
    fig=plt.figure(figsize=(16,4))
    
    s=spec.spectrum
    label = str(idx) +"):" + str(all_exposures[idx])
    
    ax1 = fig.add_subplot(1, 2, 1)
    s.plot_spectrum(ax=ax1,force_lines=True,label=label)
    
    ax2 = fig.add_subplot(1, 2, 2)
    s.plot_spectrogram(ax=ax2,scale="log")
    
    all_lambdas.append(s.lambdas)
    all_data.append(s.data)
    all_data_err.append(s.err)
    
    if FLAG_ORDER2:
        all_lambdas_order2.append(s.lambdas_order2)
        all_data_order2.append(s.data_order2)
        all_data_err_order2.append(s.err_order2)
        
    infos.append([idx,s.target.label,s.date_obs,s.airmass,s.temperature,s.pressure,s.humidity])
    idx+=1

In [14]:
infos

[[0, 'HD031128', '2022-12-09T03:49:54.625008542Z', 1.01045495027533, 0, 0, 0],
 [1, 'HD37962', '2022-12-09T02:01:33.588500419Z', 1.32181931435589, 0, 0, 0],
 [2, 'HD37962', '2022-12-09T03:15:34.256996353Z', 1.10551273278627, 0, 0, 0],
 [3, 'HD185975', '2022-12-09T01:04:11.342996723Z', 1.93628245609964, 0, 0, 0],
 [4, 'HD009051', '2022-12-09T02:33:16.154991888Z', 1.07192473453497, 0, 0, 0],
 [5, 'HD009051', '2022-12-09T02:39:19.663005039Z', 1.08159458366226, 0, 0, 0],
 [6, 'HD031128', '2022-12-09T03:49:24.033004707Z', 1.01061332768353, 0, 0, 0],
 [7, 'HD200654', '2022-12-09T01:08:23.281998723Z', 1.65427284634811, 0, 0, 0],
 [8, 'HD185975', '2022-12-09T02:20:24.959498190Z', 1.98539823658165, 0, 0, 0],
 [9, 'HD38949', '2022-12-09T01:56:02.339505700Z', 1.42536181453182, 0, 0, 0],
 [10, 'HD009051', '2022-12-09T02:39:01.845495311Z', 1.08110246037752, 0, 0, 0],
 [11, 'HD38949', '2022-12-09T03:10:58.078502228Z', 1.14680972479329, 0, 0, 0]]

In [15]:
all_lambdas

[array([ 299.23624112,  299.9462698 ,  300.65628776, ..., 1096.80411941,
        1097.48563411, 1098.16711066]),
 array([ 299.50212437,  300.21213808,  300.92214047, ..., 1096.98297554,
        1097.66440448, 1098.34579528]),
 array([ 299.30875183,  300.01881597,  300.72886895, ..., 1097.54791186,
        1098.22937216, 1098.91079428]),
 array([ 299.16979844,  299.88003779,  300.59026435, ..., 1097.42127788,
        1098.10272528, 1098.7841345 ]),
 array([ 299.39499506,  300.10519245,  300.81537884, ..., 1097.79753971,
        1098.47913423, 1099.16069055]),
 array([ 298.85177232,  299.56203334,  300.27228338, ..., 1096.65262817,
        1097.33433755, 1098.01600876]),
 array([ 298.62977364,  299.3405441 ,  300.05130397, ..., 1097.04028909,
        1097.7225092 , 1098.40469106]),
 array([ 298.96535432,  299.67565296,  300.38593865, ..., 1097.26853503,
        1097.95002642, 1098.63147964]),
 array([ 298.97856632,  299.68925585,  300.39992981, ..., 1097.41950221,
        1098.10106444, 

In [16]:
plt.imshow(spec.image.data,origin="lower")

In [17]:
plt.imshow(spec.image.data-spec.image.data.min(),origin="lower",norm=LogNorm(vmin=1,vmax=1000))

# Generate info

In [18]:
df_infos=pd.DataFrame(infos,columns=["idx","object","date_obs","airmass","temperature","pressure","humidity"])

In [19]:
df_infos.set_index('idx',inplace=True)

In [20]:
all_reftime=[ (Time(tt)-Time( df_infos["date_obs"].values[0])).to_value(u.hr) for tt in df_infos["date_obs"].values ]

In [21]:
all_reftime=np.array(all_reftime)

In [22]:
df_infos["reftime"]=all_reftime

In [23]:
NN = len(df_infos)

In [24]:
df_infos

,object,date_obs,airmass,temperature,pressure,humidity,reftime
idx,,,,,,,
0,HD031128,2022-12-09T03:49:54.625008542Z,1.010455,0,0,0,0.000000
1,HD37962,2022-12-09T02:01:33.588500419Z,1.321819,0,0,0,-1.805843
2,HD37962,2022-12-09T03:15:34.256996353Z,1.105513,0,0,0,-0.572324
3,HD185975,2022-12-09T01:04:11.342996723Z,1.936282,0,0,0,-2.762023
4,HD009051,2022-12-09T02:33:16.154991888Z,1.071925,0,0,0,-1.277353
5,HD009051,2022-12-09T02:39:19.663005039Z,1.081595,0,0,0,-1.176378
6,HD031128,2022-12-09T03:49:24.033004707Z,1.010613,0,0,0,-0.008498
7,HD200654,2022-12-09T01:08:23.281998723Z,1.654273,0,0,0,-2.692040
8,HD185975,2022-12-09T02:20:24.959498190Z,1.985398,0,0,0,-1.491574


### Save info file

In [25]:
summary_file = f"summaryspectra_{DATE}-{filterdispersername}-{configmode}-oga.csv"
df_infos.to_csv(summary_file)

## Main pickle file

In [26]:
#spec.image.target_pixcoords

In [27]:
header = spec.spectrum.header
header

SIMPLE  =                    T                                                  
TARGET  = 'HD38949 '           / name of the target in the image                
REDSHIFT=              1.1E-05 / redshift of the target                         
GRATING = 'holo4_003'          / name of the disperser                          
ROTANGLE=   0.2553919315012746 / [deg] angle of the dispersion axis             
D2CCD   =    179.9887449571588 / [mm] distance between disperser and CCD        
LSHIFT  =                  0.0                                                  
PARANGLE=   -108.7181757494001 / parallactic angle in degree                    
TARGETX =    148.5874422025884 / target position on X axis                      
TARGETY =    863.9178161975632 / target position on Y axis                      
LBDA_REF=    620.8053713364757                                                  
PIXSHIFT=  0.07321092752572178                                                  
PSF_REG =    7.2767507613976

In [28]:


all_rebin=np.zeros(NN)
all_targetx=np.zeros(NN)
all_targety=np.zeros(NN)
all_rotangle=np.zeros(NN)
all_d2ccd=np.zeros(NN)
all_pixshift=np.zeros(NN)
all_chi2_fit=np.zeros(NN)
all_a2_fit=np.zeros(NN)
all_lbda_ref=np.zeros(NN)
all_tagnumber=np.zeros(NN)
all_errors=np.zeros(NN)


for idx in range(NN):
    
    header = all_spec[idx].spectrum.header
    tagnum=str(all_num[idx])
    
    #ROTANGLE=  -0.1367006901184345 / [deg] angle of the dispersion axis             
    #D2CCD   =    179.6778868175837 / [mm] distance between disperser and CCD        
    #TARGETX =    315.9547462941386 / target position on X axis                      
    #TARGETY =    75.06785960446246 / target position on Y axis                      
    #LBDA_REF=    634.9155139280113                                                  
    #PIXSHIFT=  -0.9996981508176748                                                  
    #CHI2_FIT=    1.602115867259752                                                  
    #A2_FIT  =                  1.0                                                  
    #REBIN   =                    2 / original image rebinning factor to get spectrum
    
    try :
        rebin=header["REBIN"]
    except KeyError as e:
        rebin=2
        all_errors[idx]=1
        print(f"KeyError exception for spec {idx}: " + str(e) + f" ! ==> force rebin = {rebin}")
        
        
        
    targetx=header["TARGETX"]*rebin
    targety=header["TARGETY"]*rebin
    rotangle=header["ROTANGLE"]
    d2ccd=header["D2CCD"]
    pixshift=header["PIXSHIFT"]
    
    if header.get("CHI2_FIT") != None:
        chi2_fit=header["CHI2_FIT"]
    else:
        chi2_fit= -1
        
    if header.get("A2_FIT") != None:
        a2_fit=header["A2_FIT"]
    else:
        a2_fit=-1
    
    
    lbda_ref=header["LBDA_REF"]
    
    
    all_rebin[idx]=rebin
    all_targetx[idx]=targetx
    all_targety[idx]=targety
    all_rotangle[idx]=rotangle
    all_d2ccd[idx]=d2ccd
    all_pixshift[idx]=pixshift
    
    
    all_chi2_fit[idx]=chi2_fit
    all_a2_fit[idx]=a2_fit
    all_lbda_ref[idx]=lbda_ref
    all_tagnumber[idx]=tagnum
    


KeyError exception for spec 2: "Keyword 'REBIN' not found." ! ==> force rebin = 2
KeyError exception for spec 3: "Keyword 'REBIN' not found." ! ==> force rebin = 2
KeyError exception for spec 6: "Keyword 'REBIN' not found." ! ==> force rebin = 2
KeyError exception for spec 7: "Keyword 'REBIN' not found." ! ==> force rebin = 2
KeyError exception for spec 11: "Keyword 'REBIN' not found." ! ==> force rebin = 2


## Write pickle file

In [29]:
all_out_data = {}

for idx in range(NN):
    if FLAG_ORDER2:
        thedata = {'number':all_tagnumber[idx],
               'object':df_infos.iloc[idx]['object'],
               'dateobs':df_infos.iloc[idx]['date_obs'],
               'refhour':df_infos.iloc[idx]['reftime'],
               'airmass':df_infos.iloc[idx]['airmass'], 
               'pressure':df_infos.iloc[idx]['pressure'], 
               'temperature':df_infos.iloc[idx]['temperature'], 
               'humidity':df_infos.iloc[idx]['humidity'], 
               'targetx_pix':all_targetx[idx],
               'targety_pix':all_targety[idx],
               'rotangle':all_rotangle[idx],
               'd2ccd':all_d2ccd[idx],
               'error':all_errors[idx],    
               'all_lambdas':all_lambdas[idx],
               'all_fluxes':all_data[idx],
               'all_fluxes_err':all_data_err[idx],
               'all_lambdas_order2':all_lambdas_order2[idx],
               'all_fluxes_order2':all_data_order2[idx],
               'all_fluxes_err_order2':all_data_err_order2[idx],
              }
    else:
        thedata = {'number':all_tagnumber[idx],
               'object':df_infos.iloc[idx]['object'],
               'dateobs':df_infos.iloc[idx]['date_obs'],
               'refhour':df_infos.iloc[idx]['reftime'],
               'airmass':df_infos.iloc[idx]['airmass'],
               'pressure':df_infos.iloc[idx]['pressure'], 
               'temperature':df_infos.iloc[idx]['temperature'], 
               'humidity':df_infos.iloc[idx]['humidity'], 
               'targetx_pix':all_targetx[idx],
               'targety_pix':all_targety[idx],
               'rotangle':all_rotangle[idx],
               'd2ccd':all_d2ccd[idx],
               'error':all_errors[idx],  
               'all_lambdas':all_lambdas[idx],
               'all_fluxes':all_data[idx],
               'all_fluxes_err':all_data_err[idx],
            }         
            
    all_out_data[all_exposures[idx]]=thedata


In [30]:
#all_out_data

In [31]:
pkl_outfilename=f'run-auxtel-holo-{DATE}-{filterdispersername}-{configmode}-oga.pickle'

In [32]:
with open(pkl_outfilename, 'wb') as pickle_file:
    pickle.dump(all_out_data,pickle_file)

In [33]:
pkl_infilename=pkl_outfilename

In [34]:
with open(pkl_infilename, 'rb') as pickle_file:
    content = pickle.load(pickle_file)

In [35]:
# Get first value of dictionary
next(iter(content.items()))

(2022120800269,
 {'number': 269.0,
  'object': 'HD031128',
  'dateobs': '2022-12-09T03:49:54.625008542Z',
  'refhour': 0.0,
  'airmass': 1.01045495027533,
  'pressure': 0,
  'temperature': 0,
  'humidity': 0,
  'targetx_pix': 305.07352344410083,
  'targety_pix': 1736.167883571813,
  'rotangle': 0.15463995674873335,
  'd2ccd': 179.98755875722466,
  'error': 0.0,
  'all_lambdas': array([ 299.23624112,  299.9462698 ,  300.65628776, ..., 1096.80411941,
         1097.48563411, 1098.16711066]),
  'all_fluxes': array([-1.66300221e-16, -1.56832781e-15, -2.14033656e-15, ...,
          1.92043894e-14,  1.94140775e-14,  2.06009674e-14]),
  'all_fluxes_err': array([3.00964145e-15, 2.50408894e-15, 1.98306053e-15, ...,
         5.46568929e-16, 6.93739711e-16, 8.64746847e-16])})